<a href="https://colab.research.google.com/github/perlatomdpi/End-Point/blob/main/1_Image_Classification_App_Heroku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Image recognition - Part 1**
Create an image classifier and deply an end-point with Flask, Heroku and GitHub. <br>


In [ ]:
from flask import Flask, request, jsonify, url_for, render_template
import uuid
import os
from tensorflow.keras.models import load_model
import numpy as np
from werkzeug import secure_filename                            # save data under secure file name
from tensorflow.keras.applications import MobileNet             # pretrained NN from keras --> make prediction on images
from PIL import Image, ImageFile                                
from io import BytesIO
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input  # the pretrained network need some pre.processing such as --> colour coding, numpy array reshaping
from tensorflow.keras.applications.mobilenet import decode_predictions

In [ ]:
ALLOWED_EXTENSION = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])   # list of extantion 
IMAGE_HEIGHT =224
IMAGE_WIDTH = 224
IMAGE_CHANNELS = 3   # 3 --> rgb images
os.chdir(r'/home/utente/.../imageclass)

In [ ]:
# check if the extantion is an imaghe extantion
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.',1)[1] in ALLOWED_EXTENSION # we split the picture in the dot and [1] --> png

# **Create the Flask web application**

In [ ]:
app = Flask(__name__)
model = MobileNet(weights='imagenet', include_top=True)    # weights='imagenet' --> use a pre-trained NN weights
                                                           # include_top=True --< not retrain the NN



# end-point 1
@app.route('/index')
def index():
    return render_template('ImageML.html') # inside the imageclass folder there is the "static folder" 
                                           # and "template folder" where is store the ImageML.html --> the template of the homepage



# end-point 2 --> prediction route
@app.route('/api/image', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return render_template('ImageML.html', prediction='No posted image. Should be attribute named image')   # if not an image extension
    file = request.files['image']                                                                               # render_template is inside the css code
    
    if file.filename =='':
        return render_template('ImageML.html', prediction = 'You did not select an image')   # if nothig selected
    
    if file and allowed_file(file.filename):     # if we hava a file with allowed extension
        filename = secure_filename(file.filename)  # werkzeug --> rename the file in case there is a strange character in the filename
        print("***"+filename)                      
        x = []                                     # define an empty list
        ImageFile.LOAD_TRUNCATED_IMAGES = False    
        img = Image.open(BytesIO(file.read()))     # open the image and read it in bytes
        img.load()                                 # load the image
        img  = img.resize((IMAGE_WIDTH, IMAGE_HEIGHT), Image.ANTIALIAS)   # resize the image
        x  = image.img_to_array(img)      # convert image to an array
        x = np.expand_dims(x, axis=0)     # expand dimentions --> from the array reshape another dimension so that the array start with one        x  = preprocess_input(x)
        pred = model.predict(x)                 # predict
        lst =  decode_predictions(pred, top=3)  # and store the prediction into a list, and give me top3 prediction of the image classification
        
        items = []
        for item in lst[0]:  # lst[0] --> is the prediction
            items.append({'name': item[1], 'prob': float(item[2])}) # append to the list --> the name and probability
        
        response = {'pred': items}  # give back name and probability of the prediction
        return render_template('ImageML.html', prediction = 'I would say the image is most likely {}'.format(response))
    else:
        return render_template('ImageML.html', prediction = 'Invalid File extension')

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)  # run the app in  the debug mode